In [8]:
import sqlite3
import pandas as pd

In [37]:
class DBManager:

    def __init__(self):
        self.db_name = "task2.db"
        self.conn, self.cursor = self.connect_to_db()
        self.df_pizza_types, self.df_pizzas, self.df_orders, self.df_order_details = self.create_df()

    def connect_to_db(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        return conn, cursor
    
    @staticmethod
    def create_df():
        return pd.read_csv('pizza_types.csv', encoding='windows-1251'), pd.read_csv('pizzas.csv',  encoding='windows-1251'), pd.read_csv('orders.csv',  encoding='windows-1251'), pd.read_csv('order_details.csv',  encoding='windows-1251')

    def create_db(self):
        self.df_pizza_types.to_sql('Pizza_types', self.conn, if_exists='replace', index=False)
        self.df_pizzas.to_sql('Pizzas', self.conn, if_exists='replace', index=False)
        self.df_orders.to_sql('Orders', self.conn, if_exists='replace', index=False)
        self.df_order_details.to_sql('Order_details', self.conn, if_exists='replace', index=False)

        self.conn.commit()

    def t3(self):
        self.df_orders['time'] = pd.to_datetime(self.df_orders['time'])
        self.df_orders['hour'] = self.df_orders['time'].dt.hour
        return self.df_orders.groupby('hour')['order_id'].count().reset_index().sort_values(by='order_id', ascending = False)

    def t4(self):
        self.df_orders['date'] = pd.to_datetime(self.df_orders['date'])
        self.df_orders['month'] = self.df_orders['date'].dt.month
        seasons = {1: 'зима', 2: 'зима', 3: 'весна', 4: 'весна', 5: 'весна', 6: 'літо', 7: 'літо', 8: 'літо', 9: 'осінь', 10: 'осінь', 11: 'осінь', 12: 'зима'}
        self.df_orders['season'] = self.df_orders['month'].map(seasons)
        return self.df_orders.groupby('season')['order_id'].count().reset_index().sort_values(by='order_id', ascending = False)
    
    def t5(self):
        result = pd.merge(self.df_order_details, self.df_pizzas, on='pizza_id', how='left')
        result['sum'] = result['quantity']*result['price']
        return result.groupby('pizza_id')['sum'].sum().reset_index().sort_values(by='sum', ascending = True)
    
    def t6(self):
        self.cursor.execute("""
                WITH count_by_day AS
                    (SELECT 
                        Date, 
                        COUNT(*) AS OrdersCount
                    FROM 
                        Orders
                    GROUP BY 
                        Date)

                SELECT 
                    ROUND(AVG(OrdersCount))
                FROM 
                    count_by_day
                """)
        
        orders_by_date = self.cursor.fetchall()
        for row in orders_by_date:
            print(row[0])

    def t7(self):
        self.cursor.execute("""
                WITH count_pizza AS(
                    SELECT
                        o.order_id,
                        strftime('%m', date),
                        SUM(quantity) AS sum_pizza
                    FROM
                        Order_details od
                        LEFT JOIN Orders o ON od.order_id = o.order_id
                    WHERE
                        strftime('%m', date) = '09' 
                        or strftime('%m', date) = '10'
                        or strftime('%m', date) = '11'
                    GROUP BY
                        o.order_id
                )
                SELECT 
                    ROUND(AVG(sum_pizza))
                FROM 
                    count_pizza
                """)
        
        orders_by_date = self.cursor.fetchall()
        for row in orders_by_date:
            print(row[0])

    def t8(self):
        self.cursor.execute("""
                WITH top3 AS (
                SELECT 
                    pizza_id
                FROM 
                    Order_details od
                    LEFT JOIN Orders o ON od.order_id = o.order_id
                WHERE 
                    strftime('%m', date) = '06'
                GROUP BY
                    pizza_id
                ORDER BY
                    SUM(quantity) DESC
                LIMIT
                    3
                )
                SELECT
                    ROUND(SUM(quantity*price))
                FROM 
                    Order_details od
                    LEFT JOIN Orders o ON od.order_id = o.order_id
                    LEFT JOIN Pizzas p ON od.pizza_id = p.pizza_id
                WHERE 
                    strftime('%m', date) = '06'
                    AND
                    p.pizza_id IN (SELECT * FROM top3)
                """)
        
        orders_by_date = self.cursor.fetchall()
        for row in orders_by_date:
            print(row[0])

    def t9(self):
        self.cursor.execute("""
                SELECT
                    category,
                    size,
                    COUNT(DISTINCT order_id) AS count_orders,
                    SUM(quantity) AS count_pizzas
                FROM
                    Order_details od
                    LEFT JOIN Pizzas p ON od.pizza_id = p.pizza_id
                    LEFT JOIN Pizza_types pt ON p.pizza_type_id = pt.pizza_type_id
                GROUP BY
                    category,
                    size
                """)
        
        orders_by_date = self.cursor.fetchall()
        for row in orders_by_date:
            print(row)


   

In [38]:
manager = DBManager()

In [15]:
manager.create_db()

In [39]:
manager.t3()

,hour,order_id
3,12,2520
4,13,2455
9,18,2399
8,17,2336
10,19,2009
7,16,1920
11,20,1642
5,14,1472
6,15,1468
2,11,1231


In [40]:
manager.t4()

,season,order_id
2,літо,5549
0,весна,5492
1,зима,5210
3,осінь,5099


In [41]:
manager.t5()

,pizza_id,sum
87,the_greek_xxl,1006.60
7,calabrese_s,1212.75
13,ckn_alfredo_s,1224.00
23,green_garden_l,1923.75
43,mexicana_s,1944.00
...,...,...
3,big_meat_s,22968.00
68,spicy_ital_l,23011.75
21,four_cheese_l,23622.20
20,five_cheese_l,26066.50


In [33]:
manager.t6()

60.0


In [34]:
manager.t7()

2.0


In [35]:
manager.t8()

6411.0


In [36]:
manager.t9()

('Chicken', 'L', 4350, 4932)
('Chicken', 'M', 3521, 3894)
('Chicken', 'S', 2099, 2224)
('Classic', 'L', 3662, 4057)
('Classic', 'M', 3709, 4112)
('Classic', 'S', 5300, 6139)
('Classic', 'XL', 544, 552)
('Classic', 'XXL', 28, 28)
('Supreme', 'L', 4055, 4564)
('Supreme', 'M', 3641, 4046)
('Supreme', 'S', 3065, 3377)
('Veggie', 'L', 4722, 5403)
('Veggie', 'M', 3270, 3583)
('Veggie', 'S', 2485, 2663)
